# NOTES
Right now only takes close price. Update to include all data in the sequence.

After 3 epochs it's loss is basically non-existant. Use an early stop

Learn how to use GPU 

drop_last=True seems necessary for the data loader. The batch_size argument is expecting batches to be that size. But if the whole dataset can't be divided by the batch_size then you had some values remaining that don't fit. Could set batch_size to be some multiple if possible to minimize dropped samples 

In [386]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import torchmetrics

In [387]:
DATA_PATH = r'C:\Users\connor\PycharmProjects\trading\data\analytics\analytics_voo.csv'
df = pd.read_csv(DATA_PATH, low_memory=False)

In [388]:
df.columns

Index(['date', 'open', 'high', 'low', 'close', 'volume', 'join_date', 'Id',
       'Date', 'DateLongDescription', 'DateShortDescription', 'DayLongName',
       'DayShortName', 'MonthLongName', 'MonthShortName', 'CalendarDay',
       'CalendarWeek', 'CalendarWeekStartDateId', 'CalendarWeekEndDateId',
       'CalendarDayInWeek', 'CalendarMonth', 'CalendarMonthStartDateId',
       'CalendarMonthEndDateId', 'CalendarNumberOfDaysInMonth',
       'CalendarDayInMonth', 'CalendarQuarter', 'CalendarQuarterStartDateId',
       'CalendarQuarterEndDateId', 'CalendarQuarterStartDate',
       'CalendarNumberOfDaysInQuarter', 'CalendarDayInQuarter', 'CalendarYear',
       'CalendarYearEndDateId', 'CalendarYearStartDate',
       'CalendarNumberOfDaysInYear', 'month_join_key', 'year_join_key',
       'seven_day_ema', 'CPALTT01USM657N', 'DFF', 'EXPINF10YR', 'GDPC1',
       'RSXFS', 'T10YFF', 'UNRATE', 'macd', 'macd_signal', 'macd_hist',
       'daily_obv', 'target'],
      dtype='object')

In [389]:
non_target_columns = ['open', 'high', 'low', 'close', 'volume', 'CPALTT01USM657N', 'DFF', 'EXPINF10YR', 'GDPC1', 'RSXFS', 'T10YFF', 'UNRATE', 'macd', 'macd_signal', 'macd_hist', 'daily_obv', 'seven_day_ema', 'close', 'target']

df_cols = df[non_target_columns]

# version with just close price and date
df_target_train = df[['date', 'target']].where(df.date <= '2020-01-01')
df_target_train.dropna(inplace=True)
df_target_test = df[['date', 'target']].where(df.date > '2020-01-01')
df_target_test.dropna(inplace=True)

# version with all columns
df_target_train_v2 = df[non_target_columns].where(df.date <= '2020-01-01')
df_target_train_v2.dropna(inplace=True)
df_target_test_v2 = df[non_target_columns].where(df.date > '2020-01-01')
df_target_test_v2.dropna(inplace=True)

In [390]:
print(df_target_train.columns)
print()
print(df_target_train_v2.columns)

Index(['date', 'target'], dtype='object')

Index(['open', 'high', 'low', 'close', 'volume', 'CPALTT01USM657N', 'DFF',
       'EXPINF10YR', 'GDPC1', 'RSXFS', 'T10YFF', 'UNRATE', 'macd',
       'macd_signal', 'macd_hist', 'daily_obv', 'seven_day_ema', 'close',
       'target'],
      dtype='object')


In [391]:
def create_sequences(df, seq_length, num_rows=len(df), include_all_features=False):
    """
    Input:
    df: pandas dataframe with at least 2 columns, a date column and a target column
    seq_length: How long the sequence should be. 
    num_rows: How many rows to use. Will pull all rows unless num_rows is provided, in which case the top num_rows will be pulled.
    include_all_features: Whether to use just the target column for the sequence, or to include all features.
        
    Output:
    Generates a sequence of seq_length length. It's shape is [num_rows, seq_length] if include_all_features is False, else [num_rows, seq_length * len(df.columns)]. 
    A sequence refers to how many should be appended as columns. 
    For example, if df = [['2020-01-01',100], ['2020-01-02', 101], ['2020-01-03', 102]] and seq_length = 2, the generated sequence will be [[101,102], [102,103]]. 
    """
    
    
    xs, ys = [], []
    
    if include_all_features == False:
        for itr in range(num_rows - seq_length):
            x = df.iloc[itr:(itr + seq_length), -1]
            y = df.iloc[itr + seq_length, -1]
            xs.append(x)
            ys.append(y)
    elif include_all_features == True:
        for itr in range(num_rows - seq_length):
            x = df.iloc[itr:(itr + seq_length), :]
            y = df.iloc[itr + seq_length, -1]
            xs.append(x)
            ys.append(y)

    else:
        print(f"error: include_all_features accepts True or False, got {include_all_features} instead.")
    return np.array(xs), np.array(ys)

In [392]:
# create training set
sequence_len = 10
num_rows = 100000
X_train, y_train = create_sequences(df_target_train, sequence_len, num_rows)
X_test, y_test = create_sequences(df_target_test, sequence_len, num_rows)

X_train_v2, y_train_v2 = create_sequences(df_target_train_v2, sequence_len, num_rows, include_all_features=True)
X_test_v2, y_test_v2 = create_sequences(df_target_test_v2, sequence_len, num_rows, include_all_features=True)

In [393]:
df[non_target_columns].head(n=100)

,open,high,low,close,volume,CPALTT01USM657N,DFF,EXPINF10YR,GDPC1,RSXFS,T10YFF,UNRATE,macd,macd_signal,macd_hist,daily_obv,seven_day_ema,close,target
0,83.935,84.042,83.820,83.967,232,0.12452,0.19,1.574237,16960.864,323990.0,2.48,9.4,1.1427,1.1947,-0.0521,-148100.0,81.6086,83.967,0
1,83.888,83.995,83.743,83.889,150,0.12452,0.19,1.574237,16960.864,323990.0,2.48,9.4,1.1427,1.1947,-0.0521,-148100.0,81.6086,83.889,1
2,83.888,83.995,83.774,83.920,1600,0.12452,0.19,1.574237,16960.864,323990.0,2.48,9.4,1.1427,1.1947,-0.0521,-148100.0,81.6086,83.920,1
3,83.904,84.011,83.776,83.923,100,0.12452,0.19,1.574237,16960.864,323990.0,2.48,9.4,1.1427,1.1947,-0.0521,-148100.0,81.6086,83.923,1
4,83.966,84.073,83.851,83.998,698,0.12452,0.19,1.574237,16960.864,323990.0,2.48,9.4,1.1427,1.1947,-0.0521,-148100.0,81.6086,83.998,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,83.577,83.684,83.463,83.609,50,0.12452,0.19,1.574237,16960.864,323990.0,2.56,9.4,1.1110,1.1834,-0.0724,-165200.0,81.7122,83.609,1
96,83.608,83.715,83.494,83.640,93,0.12452,0.19,1.574237,16960.864,323990.0,2.56,9.4,1.1110,1.1834,-0.0724,-165200.0,81.7122,83.640,0
97,83.593,83.700,83.479,83.625,100,0.12452,0.19,1.574237,16960.864,323990.0,2.56,9.4,1.1110,1.1834,-0.0724,-165200.0,81.7122,83.625,1
98,83.733,83.840,83.618,83.765,100,0.12452,0.19,1.574237,16960.864,323990.0,2.56,9.4,1.1110,1.1834,-0.0724,-165200.0,81.7122,83.765,1


In [394]:
X_train[0]

array([0., 1., 1., 1., 1., 1., 0., 1., 1., 0.])

In [395]:
y_train[0]

1.0

In [396]:
X_train_v2[0]

array([[ 8.39350000e+01,  8.40420000e+01,  8.38200000e+01,
         8.39670000e+01,  2.32000000e+02,  1.24519889e-01,
         1.90000000e-01,  1.57423670e+00,  1.69608640e+04,
         3.23990000e+05,  2.48000000e+00,  9.40000000e+00,
         1.14270000e+00,  1.19470000e+00, -5.21000000e-02,
        -1.48100000e+05,  8.16086000e+01,  8.39670000e+01,
         0.00000000e+00],
       [ 8.38880000e+01,  8.39950000e+01,  8.37430000e+01,
         8.38890000e+01,  1.50000000e+02,  1.24519889e-01,
         1.90000000e-01,  1.57423670e+00,  1.69608640e+04,
         3.23990000e+05,  2.48000000e+00,  9.40000000e+00,
         1.14270000e+00,  1.19470000e+00, -5.21000000e-02,
        -1.48100000e+05,  8.16086000e+01,  8.38890000e+01,
         1.00000000e+00],
       [ 8.38880000e+01,  8.39950000e+01,  8.37740000e+01,
         8.39200000e+01,  1.60000000e+03,  1.24519889e-01,
         1.90000000e-01,  1.57423670e+00,  1.69608640e+04,
         3.23990000e+05,  2.48000000e+00,  9.40000000e+00,
    

In [397]:
y_train_v2[0]

1.0

In [398]:
print(len(X_train), len(X_train_v2))
print(len(y_train), len(y_train_v2))

99990 99990
99990 99990


# using all fields available

In [399]:
# convert df into objects Torch can read
torch_X_train  = torch.from_numpy(X_train_v2).float()
torch_y_train = torch.from_numpy(y_train_v2).float()
torch_X_test = torch.from_numpy(X_test_v2).float()
torch_y_test = torch.from_numpy(y_test_v2).float()

print(torch_X_train.shape, torch_y_train.shape)
# create test and train sets
train_data_set = TensorDataset(torch_X_train, torch_y_train)
test_data_set = TensorDataset(torch_X_test, torch_y_test)

# confirm it works
sample = train_data_set[0]
input_sample, label_sample = sample
print('input sample:', input_sample)
print('label sample:', label_sample)

torch.Size([99990, 10, 19]) torch.Size([99990])
input sample: tensor([[ 8.3935e+01,  8.4042e+01,  8.3820e+01,  8.3967e+01,  2.3200e+02,
          1.2452e-01,  1.9000e-01,  1.5742e+00,  1.6961e+04,  3.2399e+05,
          2.4800e+00,  9.4000e+00,  1.1427e+00,  1.1947e+00, -5.2100e-02,
         -1.4810e+05,  8.1609e+01,  8.3967e+01,  0.0000e+00],
        [ 8.3888e+01,  8.3995e+01,  8.3743e+01,  8.3889e+01,  1.5000e+02,
          1.2452e-01,  1.9000e-01,  1.5742e+00,  1.6961e+04,  3.2399e+05,
          2.4800e+00,  9.4000e+00,  1.1427e+00,  1.1947e+00, -5.2100e-02,
         -1.4810e+05,  8.1609e+01,  8.3889e+01,  1.0000e+00],
        [ 8.3888e+01,  8.3995e+01,  8.3774e+01,  8.3920e+01,  1.6000e+03,
          1.2452e-01,  1.9000e-01,  1.5742e+00,  1.6961e+04,  3.2399e+05,
          2.4800e+00,  9.4000e+00,  1.1427e+00,  1.1947e+00, -5.2100e-02,
         -1.4810e+05,  8.1609e+01,  8.3920e+01,  1.0000e+00],
        [ 8.3904e+01,  8.4011e+01,  8.3776e+01,  8.3923e+01,  1.0000e+02,
          1.

# Using just the close price

In [400]:
# # convert df into objects Torch can read
# torch_X_train  = torch.from_numpy(X_train).float()
# torch_y_train = torch.from_numpy(y_train).float()
# torch_X_test = torch.from_numpy(X_test).float()
# torch_y_test = torch.from_numpy(y_test).float()
# 
# # create test and train sets
# train_data_set = TensorDataset(torch_X_train, torch_y_train)
# test_data_set = TensorDataset(torch_X_test, torch_y_test)
# 
# # confirm it works
# sample = train_data_set[0]
# input_sample, label_sample = sample
# print('input sample:', input_sample)
# print('label sample:', label_sample)

In [401]:
print(input_sample.shape)

torch.Size([10, 19])


In [402]:
batch_size = 10
shuffle = True
hidden_size = 5
num_layers = 2

# create dataloader
train_dataloader = DataLoader(train_data_set, batch_size=batch_size, shuffle=shuffle, drop_last=True)
test_dataloader = DataLoader(test_data_set, batch_size=batch_size, shuffle=shuffle, drop_last=True)
# test loader
# x, y = next(iter(train_dataloader))
# 
# print('x', x, 'y', y)

In [403]:
# LTSM
class Net(nn.Module):
    def __init__(self, input_size):
        super(Net, self).__init__() #super makes all the methods available in nn.Module available for the new class Net
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True
        )
        self.fc = nn.Linear(in_features=hidden_size, out_features=1)
        self.soft_max = nn.Softmax(dim=1) # we want a binary output, not %
        
    def forward(self, x):
        h0 = torch.zeros(num_layers, x.size(0), hidden_size)
        c0 = torch.zeros(num_layers, x.size(0), hidden_size)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

In [404]:
learning_rate = 0.001
num_epochs = 50
num_features = len(df_target_train_v2.columns) # this right now is just the close date. 

net = Net(input_size=num_features)
criterion = nn.BCELoss(reduction='sum') # for binary prediction. Using the 'target' column
#criterion = nn.MSELoss() # for regression, predicting the 'close' column
optimizer = optim.Adam(
    net.parameters(), lr=learning_rate
)

for epoch in range(num_epochs):
    for seqs, labels in train_dataloader:
        seqs = seqs.view(batch_size, sequence_len, num_features)
        outputs = net(seqs).squeeze()
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item()}")
    # early break if loss isn't changing beyond the learning rate
    if loss.item() < .1:
        break

Epoch 1, Loss: 7.027163982391357
Epoch 2, Loss: 6.624287128448486
Epoch 3, Loss: 6.347145080566406
Epoch 4, Loss: 6.196864128112793
Epoch 5, Loss: 7.25331449508667
Epoch 6, Loss: 6.129487991333008
Epoch 7, Loss: 6.517644882202148
Epoch 8, Loss: 6.73011589050293
Epoch 9, Loss: 7.253875732421875
Epoch 10, Loss: 7.013100624084473
Epoch 11, Loss: 6.2647247314453125
Epoch 12, Loss: 7.514076232910156
Epoch 13, Loss: 7.620029449462891
Epoch 14, Loss: 7.017556190490723
Epoch 15, Loss: 5.976147651672363
Epoch 16, Loss: 7.243841171264648
Epoch 17, Loss: 5.879569053649902
Epoch 18, Loss: 6.194151878356934
Epoch 19, Loss: 7.025102138519287
Epoch 20, Loss: 7.543340682983398
Epoch 21, Loss: 6.554609298706055
Epoch 22, Loss: 6.763648986816406
Epoch 23, Loss: 6.485945701599121
Epoch 24, Loss: 6.905091285705566
Epoch 25, Loss: 6.5032429695129395
Epoch 26, Loss: 6.777698516845703
Epoch 27, Loss: 6.230930805206299
Epoch 28, Loss: 6.993796348571777
Epoch 29, Loss: 6.493200302124023
Epoch 30, Loss: 6.58458

In [405]:
# # Define MSE metric
# mse = torchmetrics.regression.MeanSquaredError()
# 
# net.eval()
# with torch.no_grad():
#     for seqs, labels in test_dataloader:
#         seqs = seqs.view(batch_size, torch_X_test.shape[1], num_features)
#         # Pass seqs to net and squeeze the result
#         outputs = net(seqs).squeeze()
#         mse(outputs, labels)
# 
# # Compute final metric value
# test_mse = mse.compute()
# print(f"Test MSE: {test_mse}")
# print(f"Test RMSE: {test_mse**.5}")

In [410]:
# # used for binary classifcation
# f1 = torchmetrics.F1Score(num_classes=2, task='BINARY')
# 
# net.eval()
# with torch.no_grad():
#     for seqs, labels in test_dataloader:
#         seqs = seqs.view(batch_size, sequence_len, num_features)
#         outputs = net(seqs).squeeze()
#         f1(outputs, labels)
# 
# print(f"Test F1 score: {f1.compute()}")

acc = torchmetrics.Accuracy(task="binary")

net.eval()
with torch.no_grad():
    for seqs, labels in test_dataloader:
        seqs = seqs.view(batch_size, sequence_len, num_features)
        outputs = net(seqs).squeeze()
        acc(outputs, labels)

print(f"Test accuracy score: {acc.compute()}")


Test F1 score: 0.0
Test accuracy score: 0.5077507495880127


# Results

Baseline guessing 

In [413]:
random_guess_likelihood = df.target.mean()

random_guess_likelihood

0.4687579112426077

At the minute level, 46.875% of the time the next close price is higher than the previous close price for VOO. 
The model is able to correctly predict a candle 50.077% of the time making it not much better than a random (weighted) coinflip. The thing holding the model back is that it doesn't take into account expected payoff. The average positive candel is higher than the average negative candle in absolute terms. 